In [ ]:
import random
import networkx as nx  # A library for manipulating graphs in python
from ortools.linear_solver import pywraplp
import numpy as np

random.seed(123)

In [ ]:
# Setting up our parameters

N = 50  # number of courses
p = .2  # probability of connection in the initial random graph
t = 8  # number of terms

In [ ]:
# initial random graph generation
g = nx.fast_gnp_random_graph(n=N, p=p).to_directed()
nx.set_node_attributes(g, {i: {'hours': h} for i, h in enumerate(random.choices([1, 2, 3, 4, 5], weights=[3, 1, 6, 5, 2], k=50))})

In [ ]:
# Turning the graph into a DAG by removing cycle edges one at a time
try:
    cycle = next(nx.simple_cycles(g))
    while True:
        i = random.choice(range(len(cycle)))
        g.remove_edge(cycle[i], cycle[(i+1)%len(cycle)])
        cycle = next(nx.simple_cycles(g))
except StopIteration:
    pass  # no more cycles to remove!

In [ ]:
# the optimization model
solver = pywraplp.Solver.CreateSolver('SAT')

# a num_courses-by-num_terms array of binary decision variables
a = []
for i in range(N):
    b = []
    for j in range(t):
        b.append(solver.IntVar(lb=0, ub=1, name=f'Course {i}, Term {j}'))

    a.append(b)

course_terms = np.array(a)

# add constraint: take each course only once
for i in range(N):
    solver.Add(sum(course_terms[i, :]) == 1)

# add constraint: take prereqs first (or at the same time)
for n1, n2 in g.edges:
    solver.Add(sum(course_terms[n1, :]*np.array(range(t))) <= sum(course_terms[n2, :]*np.array(range(t))))

# objective: minimize the max number of hours
max_term_hours = solver.IntVar(lb=0, ub=solver.infinity(), name=f'Max hours per term')
for i in range(t):
    solver.Add(sum(course_terms[:, i] * np.array([g.nodes[i]['hours'] for i in range(len(g.nodes))])) <= max_term_hours)

solver.Minimize(max_term_hours)

status = solver.Solve()

In [ ]:
np.vectorize(lambda x: x.solution_value())(course_terms)